In [ ]:
# 2025-4-26 run it on "python 3" base env on ins-gl-pt-gpu24-2c94136-3env-j4-l4-test-1 with 1 L4 GPU
# run on a single GPU notebook cell you can point the same script directly

In [ ]:
# 2025-5-4 confirm libcuda.so is available
# root@a085497e5485:~/mluser/git/llm-cv-pano-cubediff/cl/config# sudo find / -name "libcuda.so*" 2>/dev/null
# /usr/local/nvidia/lib64/libcuda.so
# /usr/local/nvidia/lib64/libcuda.so.1
# /usr/local/nvidia/lib64/libcuda.so.570.124.06
# /usr/local/cuda-12.4/targets/x86_64-linux/lib/stubs/libcuda.so
# /usr/local/cuda-12.4/compat/libcuda.so
# /usr/local/cuda-12.4/compat/libcuda.so.1
# /usr/local/cuda-12.4/compat/libcuda.so.550.127.08
# root@a085497e5485:~/mluser/git/llm-cv-pano-cubediff/cl/config# ldconfig -p | grep libcuda
#         libcudart.so.12 (libc6,x86-64) => /usr/local/cuda/targets/x86_64-linux/lib/libcudart.so.12
#         libcudart.so (libc6,x86-64) => /usr/local/cuda/targets/x86_64-linux/lib/libcudart.so

In [3]:
!pip install torch==2.1.2 torchvision==0.16.2 "peft>=0.8.2" \
     opencv-python==4.8.1.78 opencv-contrib-python==4.8.1.78 matplotlib==3.8.2 tqdm==4.66.1 einops==0.7.0 \
     opencv-python xformers requests pillow openexr opencv-python-headless wandb \
     bitsandbytes webdataset timm diffusers huggingface_hub accelerate

!pip uninstall torch-xla -y

  Using cached torch-2.1.2-cp310-cp310-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached torchvision-0.16.2-cp310-cp310-manylinux1_x86_64.whl.metadata (6.6 kB)
  Using cached peft-0.15.2-py3-none-any.whl.metadata (13 kB)
  Using cached opencv_python-4.8.1.78-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached opencv_contrib_python-4.8.1.78-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached matplotlib-3.8.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.8 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached einops-0.7.0-py3-none-any.whl.metadata (13 kB)
  Using cached xformers-0.0.30-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (1.0 kB)
  Using cached openexr-3.3.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached opencv_python_headless-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 k

In [ ]:
# force pip to skip its cache and grab the latest main branch
!pip install --upgrade --no-cache-dir \
    git+https://github.com/huggingface/peft.git \
    git+https://github.com/huggingface/diffusers.git


In [ ]:
!pip list | grep "dif\|hug\|trans\|torch\|acce\|peft\|bits"

In [ ]:
import os
os.environ["PYTHONPATH"] = "/home/jupyter/mluser/git/llm-cv-pano-cubediff"
os.environ["LD_LIBRARY_PATH"] = "/usr/local/nvidia/lib64:" + os.environ.get("LD_LIBRARY_PATH", "")
# Set torch compile backend
os.environ["TORCH_COMPILE_BACKEND"] = "inductor"


In [2]:
# print(f"accelerate/utils/operations.py - line 612 concatenate() - data type is {type(data)}, data[0] type is {type(data[0])}")
# if isinstance(data[0], str):
#    print(f"data[0] type is {type(data[0])}, it is {data[0]}")
# elif isinstance(data[0], dict):
#    print(f"data[0] type is {type(data[0])}, keys is {[k for k in data[0].keys()]}")
# elif isinstance(data[0], list):
#    print(f"data[0] type is {type(data[0])} size is {len(data[0])}")
#    for x in data[0]:
#        if isinstance(x, dict):
#            print(f"x type is {type(x)}, size is {len(x)}, x keys is {[k for k in x.keys()]}")
#        elif isinstance(x, torch.Tensor):
#            print(f"x type is {type(x)}, shape is {x.shape}")
#        else:
#            print(f"x type is {type(x)}, it is {x}")
# elif isinstance(data[0], torch.Tensor):
#    print(f"data[0] type is {type(data[0])}, shape is {data[0].shape}")
# else:
#    print(f"data[0] type is {type(data[0])}") 

In [ ]:
# !accelerate launch \
#   --mixed_precision=bf16 \
#   --num_processes=4 \
#   ./train_cubediff.py --cfg tiny_lora.yaml

# transparently CPU-offload part of the model or optimizer state
# that lets you push some weights or optimizer buffers to host memory.
!accelerate launch \
    --mixed_precision=bp16 \
    --num_processes=4 \
    --offload_folder=/tmp/ofa offload_optimizer=True offload_parameters=True \
    train_cubediff.py --cfg tiny_lora.yaml

In [2]:
!python ./train_cubediff.py --cfg ../config/tiny_lora.yaml


⚠️ UNet2DConditionModel.forward does not appear to be patched
⚠️ PeftModel.forward does not appear to be patched
⚠️ LoraModel.forward does not appear to be patched
⚠️ Not all classes are patched - training may encounter errors
train_cubediff.py - cfg is {'dataset': '/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/data/dataspace/polyhaven_tiny/cubediff_train.tar', 'val_dataset': '/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/data/dataspace/polyhaven_tiny/cubediff_val.tar', 'batch_size': 1, 'num_workers': 1, 'max_steps': 700, 'gradient_accum_steps': 4, 'learning_rate': 0.0001, 'lora_r': 4, 'lora_alpha': 16, 'log_every_n_steps': 20, 'sample_every_n_steps': 100, 'save_every_n_steps': 500, 'eval_every_n_steps': 500, 'use_wandb': False, 'output_dir': 'outputs/cubediff_tiny_lora', 'seed': 1337}
✅ Patched UNet2DConditionModel.forward
✅ Patched PeftModel.forward
✅ Patched LoraModel.forward
✅ Patched PeftModelForFeatureExtraction.forward
✅ PEFT and UNet classes patched successfully
Loading p

In [3]:
!pip list | grep "dif\|hug\|trans\|torch\|acce\|peft\|bits"

accelerate                               1.6.0
bitsandbytes                             0.45.5
diffusers                                0.33.1
huggingface-hub                          0.30.2
peft                                     0.15.2
torch                                    2.1.2
torchvision                              0.16.2
transformers                             4.51.3
